Usage of the package
========

First, some imports

In [1]:
import gzip
import pickle
import random
from collections import namedtuple

from territories import Territory, MissingTreeCache

## Creation of the tree

The first step is to create a tree of known entities. This can be a very compute intensive task, depending on the tree size. That is why, by default, once created, the tree is stored on disk.

Here we will create a very simple tree out of the **tree.txt** file.

In [2]:
Node = namedtuple('Node', ('id', 'parent_id', 'label', 'level'))
split = lambda x: (arg if arg != 'null' else None for arg in x[:-1].split('; '))

try:
    Territory.load_tree()
except MissingTreeCache:
    with open("tree.txt", "r") as file:
        lines = file.readlines()
        stream = ([Node(*split(x) )for x in lines])
        Territory.build_tree(data_stream=stream, save_tree=False)

Then, you can start to create territories from arbitrary territoiral units.

Entities associated territories are represented in an efficient way : if all leaves of a parent node are included in the territory, they are simply replaced by their parent node.

In [3]:
# some node of the tree
print('\n'.join([f"{e.name} | {e.partition_type}" for e in random.sample(Territory.tree.nodes(), 12)]))

Saint-Rambert-en-Bugey | COM
Prades | COM
Lesges | COM
Rochebrune | COM
Mandagout | COM
Alleyrat | COM
Mignaloux-Beauvoir | COM
Coincy | COM
Hautesvignes | COM
Beauregard-de-Terrasson | COM
Montcourt-Fromonville | COM
Bouvigny-Boyeffles | COM


In [4]:
with open("tree_large.gzip", "rb") as file:
    lines = pickle.loads(gzip.decompress(file.read()))

stream = ([Node(*split(x) )for x in lines])
Territory.build_tree(data_stream=stream)

a = Territory.from_names("COM:69123", "COM:93055", "COM:94052")
b = Territory.from_names("COM:27429", "REG:84", "DEP:75")
c = Territory.from_names("COM:38185", "COM:31555", "REG:11")
d = Territory.from_names("COM:33063", "COM:13055", "REG:28")
e = Territory.from_names("COM:35238", "COM:35047", "DEP:27")
f = Territory.from_names("COM:59350", "COM:38442", "REG:53")

You can crate a territory with names of territorial units.

In [5]:
ter = Territory.from_names("DEP:69", "COM:59350", "ARR:75106")
ter

Paris 6e|Lille|Rhône

If names are invalid, an `NotOnTree` exception will be raised.

In [6]:
try:
    Territory.from_names("DEP:69", "do not exist", "garbage")
except Exception as e:
    print(e)

do not exist, garbage where not found in the territorial tree


Territories are jsons serializable, you can simply return them from an API endpoint

In [7]:
import json

print(json.dumps(ter, indent=4))

[
    {
        "name": "Lille",
        "tu_id": "COM:59350",
        "atomic": true,
        "partition_type": "COM",
        "postal_code": null
    },
    {
        "name": "Paris 6e",
        "tu_id": "ARR:75106",
        "atomic": true,
        "partition_type": "ARR",
        "postal_code": null
    },
    {
        "name": "Rh\u00f4ne",
        "tu_id": "DEP:69",
        "atomic": false,
        "partition_type": "DEP",
        "postal_code": null
    }
]


## Operations on territories


Usual operation on territories works as expected :

In [8]:
# addition

print(a, c)
print(a + c)

Lyon|Nogent-sur-Marne|Pantin Grenoble|Toulouse|Île-de-France
Grenoble|Lyon|Toulouse|Île-de-France


In [9]:
# substraction

print(a, d)
print(a - d)

Lyon|Nogent-sur-Marne|Pantin Bordeaux|Marseille|Normandie
Lyon|Nogent-sur-Marne|Pantin


More importantly, sets operations are also supported :

In [10]:
# intersection
print(f"Intersection of {a} and {d} is {a & d}")

# union
print(f"Union of {c} and {f} is {f | c}")

Intersection of Lyon|Nogent-sur-Marne|Pantin and Bordeaux|Marseille|Normandie is {}
Union of Grenoble|Toulouse|Île-de-France and Lille|Saint-Pierre-de-Chartreuse|Bretagne is Grenoble|Lille|Saint-Pierre-de-Chartreuse|Toulouse|Bretagne|Île-de-France


Territorial units may have parents or children, but Territory do not. As a territory may be formed of several territorial units, it has a LCA, a Lowest Common Ancestor.

In [11]:
lyon_and_grenoble = Territory.from_names("COM:38185", "COM:69123")

lyon_and_grenoble.lowest_common_ancestor()

Auvergne-Rhône-Alpes

You can easily retrieve all ancestors of a territory with the `.ancestors()` method, and respectively all of its descendants with the `.descendants()` method :

In [12]:
a.ancestors()

{France,
 Île-de-France,
 Auvergne-Rhône-Alpes,
 Val-de-Marne,
 Seine-Saint-Denis,
 Rhône}

In [13]:
print(a)
a.descendants(include_itself=True)

Lyon|Nogent-sur-Marne|Pantin


{Pantin,
 Nogent-sur-Marne,
 Lyon,
 Lyon 9e,
 Lyon 8e,
 Lyon 7e,
 Lyon 6e,
 Lyon 5e,
 Lyon 4e,
 Lyon 3e,
 Lyon 2e,
 Lyon 1er}

Territories are `True` if they are not empty, but you should probably use the `is_empty()` method for clarity.

In [14]:
if Territory.from_names("DEP:69"):
    print("not empty")

if Territory.from_names():
    print("empty")

if Territory.from_names().is_empty():
    print("empty")

not empty
empty


In [15]:
from itertools import chain

with open("../test.pickle", "rb") as file:
    epci_to_comm = pickle.load(file)


Territory.from_names(*tuple(chain(*epci_to_comm.values())))

Abainville|Abaucourt-Hautecourt|Abbans-Dessous|Abbans-Dessus|Abbenans|Abbévillers|Abergement-la-Ronce|Abergement-le-Grand|Abergement-le-Petit|Abergement-lès-Thésy|Accolans|Accons|Achenheim|Achun|Achères|Adam-lès-Passavant|Adam-lès-Vercel|Adamswiller|Agen-d'Aveyron|Aghione|Aguessac|Ahéville|Aibre|Aiglepierre|Aigremont|Aigues-Vives|Aigurande|Ailhon|Ailleville|Ainay-le-Vieil|Aincreville|Aingeville|Ainvelle|Airoux|Aisy-sur-Armançon|Aix-Villemaur-Pâlis|Aixe-sur-Vienne|Aizac|Aize|Aizenay|Ajac|Ajain|Ajoux|Alaigne|Alairac|Alba-la-Romaine|Albas|Albières|Albon-d'Ardèche|Alboussière|Albé|Alet-les-Bains|Algajola|Alissas|Alièze|Allarmont|Allenc|Allenjoie|Alleyrat|Allibaudières|Alligny-Cosne|Alligny-en-Morvan|Allineuc|Allogny|Allondans|Allonnes|Allouis|Alluy|Almont-les-Junies|Altenheim|Altiani|Altier|Altorf|Altwiller|Alzonne|Alénya|Aléria|Amagney|Amance|Amancey|Amange|Amanty|Amathay-Vésigneux|Amazy|Ambacourt|Ambazac|Ambeyrac|Ambly-sur-Meuse|Ambrault|Amel-sur-l'Étang|Ameuvelle|Amirat|Amondans|Amprian

In [16]:
min(ter).partition_type

<Partition.DEP: 2>

In [17]:
ter.type

<Partition.DEP: 2>

In [28]:
from territories import Partition

s = random.sample(Territory.tree.nodes(), 1000)
ter = Territory.from_names(*(ter.tu_id for ter in s))

len([tu.tu_id for tu in ter.descendants(include_itself=True) if tu.partition_type == Partition.COM])

1155

In [31]:
import gzip
import json

# from territories import Territory




t = Territory.from_names("DEP:69")
s = json.dumps(t)

json.loads(s)

[{'name': 'Rhône',
  'tu_id': 'DEP:69',
  'atomic': False,
  'partition_type': 'DEP',
  'postal_code': None}]

In [1]:
import base64

enc = lambda s : "https://platform.datapolitics.fr/territoires/" + base64.b64encode(s.encode('utf-8')).decode('utf-8')
dec = lambda x: base64.b64decode(x.encode()).decode('utf-8').split(':')

hid = 'NjczNGE1YTBmZjlmMmFlN2NlMWIzNTljOmFydGljbGUtZG9jdW1lbnQtcHJlc3MtdjM='

dec(hid)

['6734a5a0ff9f2ae7ce1b359c', 'article-document-press-v3']

In [8]:
s = {i for i in 'azertyyuiopqsdfghjkl'}
h = iter(s)
while True:
    try:
        k = next(h)
    except StopIteration:
        break
    print(f"{k}, with len {len(s)}")
    if k == 'r':
        s.remove('p')
        s.remove('q')
        s.remove('s')
        s.remove('d')

e, with len 19
k, with len 19
i, with len 19
u, with len 19
d, with len 19
z, with len 19
o, with len 19
g, with len 19
s, with len 19
f, with len 19
r, with len 19


RuntimeError: Set changed size during iteration